In [1]:
import json
import pandas as pd
import numpy as np
from plotly.colors import hex_to_rgb

In [2]:
# PATHs

## The directory where the dune data is stored
ORI_DATA_PATH = '../data/ori'

## The directory where the three databases are stored
DATABASE_PATH = '../data/database'

## The directory where the visualization data for each visualization is stored
VIS_DATA_PATH = '../data/vis'

# ## The directory where the scraped tweets data is stored
# TWEET_PATH = '../ori_data/tweets'

In [49]:
# read transaction database
tx_db = pd.read_csv('{}/tx_db.csv'.format(DATABASE_PATH), index_col=0)

# read cryptopunk database
punk_db = pd.read_csv('{}/punk_db.csv'.format(DATABASE_PATH), index_col=0)
punk_db['attributes'] = punk_db['attributes'].apply(eval)
punk_db['attr_count_str'] = punk_db['attr_count'].apply(
        lambda x: f'{x} attributes')

address_dict = json.load(open('{}/addresses.json'.format(DATABASE_PATH)))

## Vis 1 - Sankey Diagram

In [25]:
def add_count_to_name(x, attribute_count):
    return x + ' (' + str(attribute_count[x]) + ')'

def add_count_to_names(punk_db, attribute):
    punk_db[attribute] = punk_db[attribute].apply(lambda x : add_count_to_name(x, punk_db[attribute].value_counts()))

for attribute in ['type', 'gender', 'skin_tone', 'attr_count_str']:
    add_count_to_names(punk_db, attribute)

punk_db

,type,gender,skin_tone,attr_count,attributes,skin_tone_color,img_url,attr_count_str
punk_id,,,,,,,,
0,Human (9879),Female (3840),Medium (3031),3,"[Green Eye Shadow, Earring, Blonde Bob]",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...,3 attributes (4501)
1,Human (9879),Male (6160),Dark (2824),2,"[Smile, Mohawk]",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...,2 attributes (3560)
2,Human (9879),Female (3840),Light (3006),1,[Wild Hair],#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...,1 attributes (333)
3,Human (9879),Male (6160),Dark (2824),3,"[Wild Hair, Nerd Glasses, Pipe]",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...,3 attributes (4501)
4,Human (9879),Male (6160),Medium (3031),4,"[Big Shades, Wild Hair, Earring, Goat]",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...,4 attributes (1420)
...,...,...,...,...,...,...,...,...
9995,Human (9879),Female (3840),Albino (1018),2,"[Purple Eye Shadow, Straight Hair Dark]",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...,2 attributes (3560)
9996,Human (9879),Male (6160),Light (3006),4,"[Cigarette, Earring, Crazy Hair, Smile]",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...,4 attributes (1420)
9997,Zombie (88),Male (6160),Non-human (121),2,"[Front Beard, Cap Forward]",#8DFFCD,https://www.larvalabs.com/cryptopunks/cryptopu...,2 attributes (3560)


In [26]:
def get_node_dict(punk_db):
    node_name = list()
    node_color = list()
    node_id_by_level = dict()
    
    node_count = 0
    
    # ['Human', 'Zombie', 'Ape', 'Alien']
    type_list = list(punk_db['type'].unique())
    # type_list = [f"{i} ({len(punk_db[punk_db['type']==i])})" for i in type_list]
    node_name += type_list
    node_color += ['#BD81EF', '#00916E', '#EE6123', '#FDDBC4']
    node_count += len(type_list)
    node_id_by_level['type'] = type_list
    
    # ['Female', 'Male']
    gender_list = list(punk_db['gender'].unique())
    # gender_list = [f"{i} ({len(punk_db[punk_db['gender']==i])})" for i in gender_list]
    node_name += gender_list
    node_color += ['#FF5C5C', '#5295CB']
    node_count += len(gender_list)
    node_id_by_level['gender'] = gender_list

    # ['Medium', 'Dark', 'Light', 'Albino', 'Non-human]
    skin_tone_list = list(punk_db['skin_tone'].unique())
    # skin_tone_list = [f"{i} ({len(punk_db[punk_db['skin_tone']==i])})" for i in skin_tone_list]
    node_name += skin_tone_list
    node_color += ['#DB9065', '#A4031F', '#F2A359', '#F2DC5D', '#8DFFCD']
    node_count += len(skin_tone_list)
    node_id_by_level['skin_tone'] = skin_tone_list

    # ['0 attributes', '1 attributes', '2 attributes', '3 attributes',
    # '4 attributes', '5 attributes', '6 attributes', '7 attributes']
    attr_count_list = list(punk_db['attr_count_str'].unique())
    attr_count_list = sorted(attr_count_list)
    node_name += attr_count_list
    node_color += ['#EEF2FC', '#B7E5F2', '#98DAEC',
                   '#41BBDC', '#239CBE', '#4455DA', '#2232AA', '#1B1367']
    node_count += len(attr_count_list)
    node_id_by_level['attr_count_str'] = attr_count_list

    return {'name': node_name, 'color': node_color}, node_id_by_level

In [27]:
def get_link_dict(node_df, node_name_by_level, punk_db):
    source_li = list()
    target_li = list()
    value_li = list()
    link_color_li = list()
    
    levels_list = ['type', 'gender', 'skin_tone', 'attr_count_str']
    
    # links between each pair of neighbor levels
    for level_idx in range(len(levels_list)-1):
        source_level = levels_list[level_idx]
        target_level = levels_list[level_idx+1]
        
        for source_node_name in node_name_by_level[source_level]:
            source_node_id = node_df.loc[source_node_name, 'index']
            for target_node_name in node_name_by_level[target_level]:
                target_node_id = node_df.loc[target_node_name, 'index']
                
                value = punk_db[(punk_db[source_level] == source_node_name) & (
                    punk_db[target_level] == target_node_name)].shape[0]
                
                link_color = convert_to_rgba(node_df.loc[target_node_name, 'color'])
                
                source_li.append(str(source_node_id))
                target_li.append(str(target_node_id))
                value_li.append(value)
                link_color_li.append(link_color)

    return {'source': source_li, 'target': target_li, 'value': value_li, 'color': link_color_li}

In [28]:
def between_skin_tone_and_attributes(item, source_skin_tone, target_attribute):
    return (item['skin_tone'] == source_skin_tone) and (target_attribute in item['attributes'])


def convert_to_rgba(hex_color, a=0.5):
    return 'rgba({},{},{},{})'.format(*hex_to_rgb(hex_color), a)


def get_dataset_vis1(punk_db):
    dataset_vis1 = dict()

    # node
    node, node_name_by_level = get_node_dict(punk_db)
    
    # Save node data
    dataset_vis1['node'] = node
    print('{} nodes are saved.'.format(len(node['name'])))
    
    
    # link
    node_df = pd.DataFrame.from_dict(node)
    node_df.reset_index(inplace=True)
    node_df.set_index('name', inplace=True)

    link = get_link_dict(node_df, node_name_by_level, punk_db)
    
    # Save link data
    dataset_vis1['link'] = link
    print('{} links are saved.'.format(len(link['source'])))

    return dataset_vis1


vis1_data = get_dataset_vis1(punk_db)
with open('{}/vis1_data.json'.format(VIS_DATA_PATH), 'w') as f:
    json.dump(vis1_data, f)


19 nodes are saved.
58 links are saved.


## Vis2 

In [46]:
def set_bubble_color(x):
    if x == 'Dark':
        return '#933e18'
    elif x == 'Medium':
        return '#dc5d24'
    elif x == 'Light':
        return '#f6773d'
    else:
        return '#f9a47e'


def get_dataset_vis3(tx_db, punk_db, address_dict):
    vis3_df = tx_db.copy(deep=True)

    vis3_df = vis3_df.loc[vis3_df['date'] > '2020-12-31']

    # drop the punk transfer transactions
    vis3_df.dropna(axis=0, inplace=True)

    # set punk_skin_color as bubble color
    vis3_df['punk_skin_color'] = vis3_df['punk_id'].apply(
        lambda x: punk_db.loc[x, 'skin_tone'])
    vis3_df['bubble_color'] = vis3_df['punk_skin_color'].apply(
        set_bubble_color)

    # set punk_attr_count as bubble size
    vis3_df['punk_attr_count'] = vis3_df['punk_id'].apply(
        lambda x: punk_db.loc[x, 'attr_count'])
    vis3_df.rename(
        columns={'punk_attr_count': 'bubble_size', 'eth_price': 'price'}, inplace=True)

    # set punk features
    vis3_df['img_url'] = vis3_df['punk_id'].apply(
        lambda x: punk_db.loc[x, 'img_url'])

    vis3_df['punk_skin_tone'] = vis3_df['punk_id'].apply(
        lambda x: punk_db.loc[x, 'skin_tone'])
    vis3_df['punk_skin_tone'] = vis3_df['punk_skin_tone'].apply(
        lambda x: 'Non-human' if pd.isna(x) else x)

    # set trader addresses
    vis3_df['from_address'] = vis3_df['from'].apply(lambda x: address_dict[str(x)])
    vis3_df['to_address'] = vis3_df['to'].apply(lambda x: address_dict[str(x)])
    vis3_df = vis3_df.loc[:, ['date',  'price', 'bubble_size', 'bubble_color', 'from_address', 'to_address', 'punk_id', 'img_url', 'punk_skin_tone']]
    # price75 = vis3_df['price'].describe()['75%']
    vis3_df = vis3_df[(vis3_df['price'] < 300) & (vis3_df['price'] > 0)]
    return vis3_df


vis3_df = get_dataset_vis3(tx_db, punk_db, address_dict)
vis3_df.to_csv('{}/vis3_data.csv'.format(VIS_DATA_PATH))
vis3_df

,date,price,bubble_size,bubble_color,from_address,to_address,punk_id,img_url,punk_skin_tone
17,2021-08-28,180.00,3,#dc5d24,\x7132c9f36abe62eab74cdfdd08c154c9ae45691b,\x8f10ae3ffd993bfb0a71e58e1bb606439dd5b301,5621,https://www.larvalabs.com/cryptopunks/cryptopu...,Medium
34,2021-02-22,30.00,4,#f6773d,\x052564eb0fd8b340803df55def89c25c432f43f4,\xfd6bf7971c58cb92d14d226cb7840b82f585c53c,8781,https://www.larvalabs.com/cryptopunks/cryptopu...,Light
35,2021-03-14,35.00,4,#f6773d,\xfd6bf7971c58cb92d14d226cb7840b82f585c53c,\x1455f0b4c58906463f87aba9e84f101124926043,8781,https://www.larvalabs.com/cryptopunks/cryptopu...,Light
36,2021-04-09,27.00,4,#f6773d,\xe907c99288007b394e77267f9b77e1053cd55d29,\x7c0f4b0fc6881c33b5854dac2ec767b87debd078,8781,https://www.larvalabs.com/cryptopunks/cryptopu...,Light
37,2022-04-15,69.00,4,#f6773d,\x8b6fede39b04c23bfcc7fe74e2680139a68bdbd7,\xccf2075c74de218cdd79b1ef699a2a31e2940f83,8781,https://www.larvalabs.com/cryptopunks/cryptopu...,Light
...,...,...,...,...,...,...,...,...,...
17821,2022-07-17,88.88,2,#dc5d24,\x6f4a2d3a4f47f9c647d86c929755593911ee91ec,\xc88345ba0b66d140b8c0e27fa0fec1379036ffdd,4656,https://www.larvalabs.com/cryptopunks/cryptopu...,Medium
17822,2022-07-17,88.00,3,#dc5d24,\xa5f6a8db3d35312e428c7b462a2cc0dffa11fc98,\xb5696e4057b9ba76616cecb5a537eaca7b3cdf54,4749,https://www.larvalabs.com/cryptopunks/cryptopu...,Medium
17823,2022-07-18,84.00,3,#f6773d,\x51ec89f1fcfed8c69a1b0865a7550ece0677cf5f,\x51eac3daa1c34f5c2874aa62097ac9965a180b6d,4569,https://www.larvalabs.com/cryptopunks/cryptopu...,Light
17824,2022-07-21,86.68,3,#933e18,\x20481b79a4f03b624d214d23adf5bf5f33beb4aa,\xac1e9f1f2413c594df0da2903e93b08ab0fba0ee,4755,https://www.larvalabs.com/cryptopunks/cryptopu...,Dark


## Vis 3

In [47]:
tx_db

,date,from,to,eth_price,punk_id,type,gender,skin_tone,attr_count,attributes,skin_tone_color,img_url
0,2017-06-23,1406,1218,0.03,6548,Human,Male,Albino,4,"['Front Beard', 'Earring', 'Do-rag', 'Clown Ey...",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
1,2017-07-03,1218,3828,0.49,6548,Human,Male,Albino,4,"['Front Beard', 'Earring', 'Do-rag', 'Clown Ey...",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
2,2017-06-23,1406,1218,0.04,5719,Human,Female,Medium,3,"['Welding Goggles', 'Dark Hair', 'Cigarette']",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
3,2019-03-20,920,1472,0.99,5719,Human,Female,Medium,3,"['Welding Goggles', 'Dark Hair', 'Cigarette']",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
4,2017-06-23,1406,2518,0.25,1841,Human,Female,Dark,3,"['Black Lipstick', 'Green Eye Shadow', 'Mohawk...",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...
...,...,...,...,...,...,...,...,...,...,...,...,...
17821,2022-07-17,2292,2070,88.88,4656,Human,Female,Medium,2,"['Blonde Short', 'Classic Shades']",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
17822,2022-07-17,5922,916,88.00,4749,Human,Male,Medium,3,"['Frown', 'Do-rag', 'Small Shades']",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
17823,2022-07-18,107,1044,84.00,4569,Human,Male,Light,3,"['Clown Eyes Blue', 'Shadow Beard', 'Mohawk']",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
17824,2022-07-21,244,801,86.68,4755,Human,Male,Dark,3,"['Messy Hair', 'Earring', 'Normal Beard']",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...


In [ ]:
tx_db